In [31]:
library("tidyverse")

In [ ]:
apex factor \
 --out /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Factor_and_Covariate/AC.APEX \
 --iter 1000 \
 --cov "/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov" \
 --factors 4 \
 --bed /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Phenotype/AC.mol_phe.bed.gz

In [ ]:
apex factor \
 --out /mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Factor_and_Covariate/AC.APEX \
 --iter 1000 \
 --cov "/home/hs3163/GIT/ADSPFG-xQTL/MWE/MWE.cov" \
 --factors 4 \
 --bed /mnt/mfs/statgen/xqtl_workflow_testing/testing_complete/Data_Processing/Phenotype/AC.mol_phe.bed.gz

In [ ]:
phenotype_df = covariates_df[np.intersect1d(phenotype_df.columns, covariates_df.index)]

In [32]:
library("igraph")

In [68]:
kin0 <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.kin0', header=F, stringsAsFactor=F)
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")

In [69]:
# Remove related individuals while keeping maximum number of individuals
# this function is simplified from:
# https://rdrr.io/cran/plinkQC/src/R/utils.R
#' @param relatedness [data.frame] containing pair-wise relatedness estimates
#' (in column [relatednessRelatedness]) for individual 1 (in column
#' [relatednessID1] and individual 2 (in column [relatednessID1]). Columns
#' relatednessID1, relatednessID2 and relatednessRelatedness have to present,
#' while additional columns such as family IDs can be present. Default column
#' names correspond to column names in output of plink --genome
#' (\url{https://www.cog-genomics.org/plink/1.9/ibd}). All original
#' columns for pair-wise highIBDTh fails will be returned in fail_IBD.
#' @param relatednessTh [double] Threshold for filtering related individuals.
#' Individuals, whose pair-wise relatedness estimates are greater than this
#' threshold are considered related.
relatednessFilter <- function(relatedness,
                              relatednessTh,
                              relatednessID1="ID1",
                              relatednessID2="ID2",
                              relatednessRelatedness="KINSHIP") {
    # format data
    if (!(relatednessID1 %in% names(relatedness))) {
        stop(paste("Column", relatednessID1, "for relatedness not found!"))
    }
    if (!(relatednessID2 %in% names(relatedness))) {
        stop(paste("Column", relatednessID1, "for relatedness not found!"))
    }
    if (!(relatednessRelatedness %in% names(relatedness))) {
        stop(paste("Column", relatednessRelatedness,
                   "for relatedness not found!"))
    }

    ID1_index <- which(colnames(relatedness) == relatednessID1)
    ID2_index <- which(colnames(relatedness) == relatednessID2)

    relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
    relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

    relatedness_names <- names(relatedness)
    names(relatedness)[ID1_index] <- "ID1"
    names(relatedness)[ID2_index] <- "ID2"
    names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

    # Remove symmetric IID rows
    relatedness_original <- relatedness
    relatedness <- dplyr::select(relatedness, ID1, ID2, M)

    sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
        c(sort(c(pair[1], pair[2])))
        })), stringsAsFactors=FALSE)
    keepIndex <- which(!duplicated(sortedIDs))

    relatedness_original <- relatedness_original[keepIndex,]
    relatedness <- relatedness[keepIndex,]

    # individuals with at least one pair-wise comparison > relatednessTh
    # return NULL to failIDs if no one fails the relatedness check
    highRelated <- dplyr::filter(relatedness, M > relatednessTh)
    if (nrow(highRelated) == 0) {
        return(list(relatednessFails=NULL, failIDs=NULL))
    }

    # all samples with related individuals
    allRelated <- c(highRelated$ID1, highRelated$ID2)
    uniqueIIDs <- unique(allRelated)

    # Further selection of samples with relatives in cohort
    multipleRelative <- unique(allRelated[duplicated(allRelated)])
    singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

    highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                        highRelated$ID2 %in% multipleRelative,]
    highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                       highRelated$ID2 %in% singleRelative,]

    # Only one related samples per individual
    if(length(singleRelative) != 0) {
      # randomly choose one to exclude
      failIDs_single <- highRelatedSingle[,1]

    } else {
      failIDs_single <- NULL
    }

    # An individual has multiple relatives
    if(length(multipleRelative) != 0) {
        relatedPerID <- lapply(multipleRelative, function(x) {
            tmp <- highRelatedMultiple[rowSums(
                cbind(highRelatedMultiple$ID1 %in% x,
                      highRelatedMultiple$ID2 %in% x)) != 0,1:2]
            rel <- unique(unlist(tmp))
            return(rel)
        })
        names(relatedPerID) <- multipleRelative

        keepIDs_multiple <- lapply(relatedPerID, function(x) {
            pairwise <- t(combn(x, 2))
            index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                          highRelatedMultiple$ID2 %in% pairwise[,2]) |
                (highRelatedMultiple$ID1 %in% pairwise[,2] &
                     highRelatedMultiple$ID2 %in% pairwise[,1])
            combination <- highRelatedMultiple[index,]
            combination_graph <- igraph::graph_from_data_frame(combination,
                                                               directed=FALSE)
            all_iv_set <- igraph::ivs(combination_graph)
            length_iv_set <- sapply(all_iv_set, function(x) length(x))

            if (all(length_iv_set == 1)) {
                # check how often they occurr elsewhere
                occurrence <- sapply(x, function(id) {
                    sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                })
                # if occurrence the same everywhere, pick the first, else keep
                # the one with minimum occurrence elsewhere
                if (length(unique(occurrence)) == 1) {
                    nonRelated <- sort(x)[1]
                } else {
                    nonRelated <- names(occurrence)[which.min(occurrence)]
                }
            } else {
                nonRelated <- all_iv_set[which.max(length_iv_set)]
            }
            return(nonRelated)
        })
        keepIDs_multiple <- unique(unlist(keepIDs_multiple))
        failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                   keepIDs_multiple])
    } else {
        failIDs_multiple <- NULL
    }
    allFailIIDs <- c(failIDs_single, failIDs_multiple)
    relatednessFails <- lapply(allFailIIDs, function(id) {
        fail_inorder <- relatedness_original$ID1 == id &
            relatedness_original$M > relatednessTh
        fail_inreverse <- relatedness_original$ID2 == id &
            relatedness_original$M > relatednessTh
        if (any(fail_inreverse)) {
            inreverse <- relatedness_original[fail_inreverse, ]
            id1 <- ID1_index
            id2 <- ID2_index
            inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
            names(inreverse) <- relatedness_names
        } else {
            inreverse <- NULL
        }
        inorder <- relatedness_original[fail_inorder, ]
        names(inorder) <- relatedness_names
        return(rbind(inorder, inreverse))
    })
    relatednessFails <- do.call(rbind, relatednessFails)
    if (nrow(relatednessFails) == 0) {
        relatednessFails <- NULL
        failIDs <- NULL
    } else {
        names(relatednessFails) <- relatedness_names
        rownames(relatednessFails) <- 1:nrow(relatednessFails)
        uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
        failIDs <- uniqueFails[,ID1_index]
    }
    return(list(relatednessFails=relatednessFails, failIDs=failIDs))
}


# main code


[1] 1


In [67]:
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")

In [145]:
LD_Recipe = "~/GIT/ADSPFG-xQTL/MWE/LD_Recipe"

In [153]:
a$ld.table = read_delim(LD_Recipe , "\t")%>%mutate(ld.path = paste0(ld_file_prefix, gene_ID ,ld_file_surfix )) 


── Column specification ────────────────────────────────────────────────────────
cols(
  ld_file_prefix = col_character(),
  ld_file_surfix = col_character(),
  Theme = col_character()
)




In [1]:
a

ERROR: Error in eval(expr, envir, enclos): object 'a' not found


In [156]:
name = c("PCC","AC-DLPFC")

In [164]:
rownames(a$ld.table ) = c("PCC","AC-DLPFC")

Warning message:
“Setting row names on a tibble is deprecated.”


In [151]:
paste0(a$ld.table$ld_file_prefix,a$ld.table$ld_file_surfix)

[1] "/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename..merged.ld.rds"
[2] "/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename..merged.ld.rds"

In [ ]:
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
if (TRUE) {
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")$failIDs
    tmp1 <- kin0[,1:2]
    tmp2 <- kin0[,3:4]
    colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
    # Get the family ID for these rels so there are two columns FID and IID in the output
    lookup <- dplyr::distinct(rbind(tmp1,tmp2))
    dat <- lookup[which(lookup[,2] %in% rel),]
} else {
    rel <- kin0 %>% filter(KINSHIP >= 0.0625)
    IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
    dat <- data.frame(IID)
    dat <- dat %>%
        mutate(FID = IID) %>%
        select(FID, IID)
}
cat("There are", nrow(dat),"related individuals using a kinship threshold of 0.0625\n")
write.table(dat,'/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.tmp.related_id', quote=FALSE, row.names=FALSE, col.names=FALSE)

In [61]:
colnames(kin0) <- c("FID1","ID1","FID2","ID2","NSNP","HETHET","IBS0","KINSHIP")
if (TRUE) {
    rel <- relatednessFilter(kin0, 0.0625, "ID1", "ID2", "KINSHIP")$failIDs
    tmp1 <- kin0[,1:2]
    tmp2 <- kin0[,3:4]
    colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
    # Get the family ID for these rels so there are two columns FID and IID in the output
    lookup <- dplyr::distinct(rbind(tmp1,tmp2))
    dat <- lookup[which(lookup[,2] %in% rel),]
} 

In [63]:
relatednessFilter <- function(relatedness, 
                                  relatednessTh,
                                  relatednessID1="ID1", 
                                  relatednessID2="ID2",
                                  relatednessRelatedness="KINSHIP") {
        # format data
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

        relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"
        print(1)
        # Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1, ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]
        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }

        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]
        print(2)
        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
        print(3)
        # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }
        print(4)
        return(list(relatednessFails=relatednessFails, failIDs=failIDs))
    }
    

In [81]:
        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
        print(3)
       
    

[1] 3


In [84]:
if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })

ERROR: Error in parse(text = x, srcfile = src): <text>:9:0: unexpected end of input
7:                 return(rel)
8:             })
  ^


In [86]:
      relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })

In [2]:
a = readRDS("/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Fine_mapping/preprocessed/ENSG00000007944.rds")

In [3]:
a$

PCC,AC-DLPFC
0.00697397,0.0094
0.00999229,0.0096
0.00637934,0.0097
-0.02060820,0.0120
0.02724140,0.0315
-0.02795820,0.0106
-0.05124390,0.0242
0.00999229,0.0096
0.00999229,0.0096
-0.02817700,0.0106


In [130]:
paste("ld_file_prefix","gene_id","ld_file_surfix",collapse   = "")

[1] "ld_file_prefixgene_idld_file_surfix"

In [90]:
 keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))})

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })

In [91]:
  lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))})

In [93]:
test = function(x){
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))}

In [97]:
                pairwise <- t(combn(x, 2))

In [99]:
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])

In [103]:
                combination <- highRelatedMultiple[index,]


In [105]:
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)

In [107]:
all_iv_set <- igraph::ivs(combination_graph)

In [116]:
length(all_iv_set[[6]])

[1] 1

In [95]:
x = relatedPerID[[1]]

In [83]:
 # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }


In [51]:
        rel <- relatednessFilter(kin0, ${kinship}, "ID1", "ID2", "KINSHIP")$failIDs
        tmp1 <- kin0[,1:2]
        tmp2 <- kin0[,3:4]
        colnames(tmp1) = colnames(tmp2) = c("FID", "ID")
        # Get the family ID for these rels so there are two columns FID and IID in the output
        lookup <- dplyr::distinct(rbind(tmp1,tmp2))
        dat <- lookup[which(lookup[,2] %in% rel),]
    } else {
        rel <- kin0 %>% filter(KINSHIP >= ${kinship})
        IID <- sort(unique(unlist(rel[, c("ID1", "ID2")])))
        dat <- data.frame(IID)
        dat <- dat %>%
            mutate(FID = IID) %>%
            select(FID, IID)
    }
    cat("There are", nrow(dat),"related individuals using a kinship threshold of ${kinship}\n")
    write.table(dat,${_output:r}, quote=FALSE, row.names=FALSE, col.names=FALSE)

There are 0 related individuals using a kinship threshold of ${kinship}


In [56]:
    write.table(dat,"~/empty_related", quote=FALSE, row.names=FALSE, col.names=FALSE)

In [70]:
relatedness = kin0
                            relatednessTh = 0.0625
                                  relatednessID1="ID1"  
                                  relatednessID2="ID2" 
                                  relatednessRelatedness="KINSHIP"

In [71]:
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

In [72]:
 relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

In [73]:
# Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1,ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]

In [75]:
length(keepIndex)

[1] 631

In [76]:

        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }


In [119]:
if(F){}

In [78]:


        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]


In [14]:
# Remove symmetric IID rows
        relatedness_original <- relatedness


In [13]:
relatedness <- dplyr::select(relatedness, ID1,ID2,M)

In [29]:
 
        # format data
        if (!(relatednessID1 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessID2 %in% names(relatedness))) {
            stop(paste("Column", relatednessID1, "for relatedness not found!"))
        }
        if (!(relatednessRelatedness %in% names(relatedness))) {
            stop(paste("Column", relatednessRelatedness,
                       "for relatedness not found!"))
        }

        ID1_index <- which(colnames(relatedness) == relatednessID1)
        ID2_index <- which(colnames(relatedness) == relatednessID2)

        relatedness[,ID1_index] <- as.character(relatedness[,ID1_index])
        relatedness[,ID2_index] <- as.character(relatedness[,ID2_index])

        relatedness_names <- names(relatedness)
        names(relatedness)[ID1_index] <- "ID1"
        names(relatedness)[ID2_index] <- "ID2"
        names(relatedness)[names(relatedness) == relatednessRelatedness] <- "M"

        # Remove symmetric IID rows
        relatedness_original <- relatedness
        relatedness <- dplyr::select(relatedness, ID1,ID2, M)

        sortedIDs <- data.frame(t(apply(relatedness, 1, function(pair) {
            c(sort(c(pair[1], pair[2])))
            })), stringsAsFactors=FALSE)
        keepIndex <- which(!duplicated(sortedIDs))

        relatedness_original <- relatedness_original[keepIndex,]
        relatedness <- relatedness[keepIndex,]

        # individuals with at least one pair-wise comparison > relatednessTh
        # return NULL to failIDs if no one fails the relatedness check
        highRelated <- dplyr::filter(relatedness, M > relatednessTh)
        if (nrow(highRelated) == 0) {
            return(list(relatednessFails=NULL, failIDs=NULL))
        }

        # all samples with related individuals
        allRelated <- c(highRelated$ID1, highRelated$ID2)
        uniqueIIDs <- unique(allRelated)

        # Further selection of samples with relatives in cohort
        multipleRelative <- unique(allRelated[duplicated(allRelated)])
        singleRelative <- uniqueIIDs[!uniqueIIDs %in% multipleRelative]

        highRelatedMultiple <- highRelated[highRelated$ID1 %in% multipleRelative |
                                            highRelated$ID2 %in% multipleRelative,]
        highRelatedSingle <- highRelated[highRelated$ID1 %in% singleRelative &
                                           highRelated$ID2 %in% singleRelative,]

        # Only one related samples per individual
        if(length(singleRelative) != 0) {
          # randomly choose one to exclude
          failIDs_single <- highRelatedSingle[,1]
            
        } else {
          failIDs_single <- NULL
        }
  
        # An individual has multiple relatives
        if(length(multipleRelative) != 0) {
            relatedPerID <- lapply(multipleRelative, function(x) {
                tmp <- highRelatedMultiple[rowSums(
                    cbind(highRelatedMultiple$ID1 %in% x,
                          highRelatedMultiple$ID2 %in% x)) != 0,1:2]
                rel <- unique(unlist(tmp))
                return(rel)
            })
            names(relatedPerID) <- multipleRelative

            keepIDs_multiple <- lapply(relatedPerID, function(x) {
                pairwise <- t(combn(x, 2))
                index <- (highRelatedMultiple$ID1 %in% pairwise[,1] &
                              highRelatedMultiple$ID2 %in% pairwise[,2]) |
                    (highRelatedMultiple$ID1 %in% pairwise[,2] &
                         highRelatedMultiple$ID2 %in% pairwise[,1])
                combination <- highRelatedMultiple[index,]
                combination_graph <- igraph::graph_from_data_frame(combination,
                                                                   directed=FALSE)
                all_iv_set <- igraph::ivs(combination_graph)
                length_iv_set <- sapply(all_iv_set, function(x) length(x))

                if (all(length_iv_set == 1)) {
                    # check how often they occurr elsewhere
                    occurrence <- sapply(x, function(id) {
                        sum(sapply(relatedPerID, function(idlist) id %in% idlist))
                    })
                    # if occurrence the same everywhere, pick the first, else keep
                    # the one with minimum occurrence elsewhere
                    if (length(unique(occurrence)) == 1) {
                        nonRelated <- sort(x)[1]
                    } else {
                        nonRelated <- names(occurrence)[which.min(occurrence)]
                    }
                } else {
                    nonRelated <- all_iv_set[which.max(length_iv_set)]
                }
                return(nonRelated)
            })
            keepIDs_multiple <- unique(unlist(keepIDs_multiple))
            failIDs_multiple <- c(multipleRelative[!multipleRelative %in%
                                                       keepIDs_multiple])
        } else {
            failIDs_multiple <- NULL
        }
        allFailIIDs <- c(failIDs_single, failIDs_multiple)
        relatednessFails <- lapply(allFailIIDs, function(id) {
            fail_inorder <- relatedness_original$ID1 == id &
                relatedness_original$M > relatednessTh
            fail_inreverse <- relatedness_original$ID2 == id &
                relatedness_original$M > relatednessTh
            if (any(fail_inreverse)) {
                inreverse <- relatedness_original[fail_inreverse, ]
                id1 <- ID1_index
                id2 <- ID2_index
                inreverse[,c(id1, id2)] <- inreverse[,c(id2, id1)]
                names(inreverse) <- relatedness_names
            } else {
                inreverse <- NULL
            }
            inorder <- relatedness_original[fail_inorder, ]
            names(inorder) <- relatedness_names
            return(rbind(inorder, inreverse))
        })
        relatednessFails <- do.call(rbind, relatednessFails)
        if (nrow(relatednessFails) == 0) {
            relatednessFails <- NULL
            failIDs <- NULL
        } else {
            names(relatednessFails) <- relatedness_names
            rownames(relatednessFails) <- 1:nrow(relatednessFails)
            uniqueFails <- relatednessFails[!duplicated(relatednessFails[,ID1_index]),]
            failIDs <- uniqueFails[,ID1_index]
        }

$relatednessFails
NULL

$failIDs
NULL

ERROR: Error in if (nrow(relatednessFails) == 0) {: argument is of length zero


In [30]:
relatednessFails


NULL

In [22]:
relatednessRelatedness

[1] "KINSHIP"

In [23]:
relatedness

FID1,ID1,FID2,ID2,NSNP,HETHET,IBS0,M
<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>
ROS20998065,ROS20998065,MAP50103679,MAP50103679,260130,0.0984700,3.28605e-02,0.0628394
ROS79590778,ROS79590778,ROS20376029,ROS20376029,260353,0.0962386,3.17185e-02,0.0625368
SM-CJEFR,SM-CJEFR,ROS20376029,ROS20376029,260480,0.2451900,7.29423e-05,0.4940130
SM-CJEFZ,SM-CJEFZ,MAP50402431,MAP50402431,260442,0.2333960,8.44718e-05,0.4934330
SM-CJEG8,SM-CJEG8,MAP50103679,MAP50103679,260243,0.1007370,3.41489e-02,0.0635570
SM-CJEG8,SM-CJEG8,ROS20998065,ROS20998065,260146,0.1035800,3.34082e-02,0.0707883
SM-CJEGZ,SM-CJEGZ,ROS10524640,ROS10524640,260325,0.2473020,6.53030e-05,0.4952050
SM-CJEHD,SM-CJEHD,ROS20376029,ROS20376029,260295,0.0948078,2.88711e-02,0.0703430
SM-CJEHD,SM-CJEHD,SM-CJEFR,SM-CJEFR,260336,0.0946162,2.89357e-02,0.0697255


In [ ]:
# Load required libraries
library(dplyr)
library(flashpcaR)
# Read the PLINK binary files
frel <- '/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.unrelated'
# Read loadings, center and scale from previous PCA
dat <- readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.exp.pca.rds')
f <- dat$pca_model
# compute centroids before projecting back the samples
# (calculate mean/median/cov per pop)
if(TRUE){
    pop_group <- split(dat$pc_scores[ ,c(paste0("PC_", 1:5))], list(Group = dat$pc_scores$RACE))
    dat$pc_cov <- lapply(pop_group, function(x) cov(x))
    dat$pc_mean <- lapply(pop_group, function(x) sapply(x, mean))
    dat$pc_median <- lapply(pop_group, function(x) sapply(x, median))
} else {
    dat$pc_cov <- cov(f$projection[,1:5])
    dat$pc_mean <- apply(f$projection[,1:5], 2, mean)
    dat$pc_median <- apply(f$projection[,1:5], 2, median)
}

# Read the bim file to obtain reference alleles
bim <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.AC.unrelated.bim',stringsAsFactors =F)
ref <- as.character(bim[,5])
names(ref) <- bim[,2]
overlapped_variants <- match(names(ref), rownames(f$loadings))
# Project related samples
fpro <- project(frel, loadings=f$loadings[overlapped_variants,], orig_mean=f$center[overlapped_variants], orig_sd=f$scale[overlapped_variants], ref_allele=ref)
pca <- fpro$projection
k = 10
pca <- as.data.frame(fpro$projection)
pca <- tibble::rownames_to_column(pca, "ID")
colnames(pca) <- c("ID",paste0("PC_", 1:k))
pca$IID <- sapply(strsplit(as.character(pca$ID),':'), "[", 1)
# Read fam file with phenotypes
pheno <- read.table('/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.exp', sep="\t", header=T, stringsAsFactors =F)
pca <-merge(pheno, pca, by="IID", all=FALSE)
dat$pc_scores = rbind(dat$pc_scores, pca)

# Write the PC scores to a file
write.table(dat$pc_scores,"/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.AC.pca.projected.txt", sep="\t", quote=FALSE, row.names=FALSE, col.names=TRUE)
# save results
saveRDS(dat, '/mnt/mfs/statgen/xqtl_workflow_testing/testing_1/Data_Processing/Genotype/PCA/AC.mol_phe.bed.AC.pca.projected.rds')

In [8]:
get_prior_indices <- function(Z, U) {
  # make sure the prior col/rows match the colnames of the Y matrix
  z_names = colnames(Z)
  u_names = colnames(U)
  if (is.null(z_names) || is.null(u_names)) {
      return(NULL)
  } else if (identical(z_names, u_names)) {
      return(NULL)
  } else {
      return(match(z_names, u_names))
  }
}

library(mvsusieR)
library("dplyr")
library("tibble")
library("purrr")
library("readr")
library("tidyr")
dat = readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Fine_mapping/preprocessed/ENSG00000010818.rds')
gene_id = "ENSG00000010818"




if (FALSE == TRUE){
  resid_cor = paste(c("/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/residcor_2/",gene_id),collapse = "")
  }else{resid_cor = '/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Meta_analysis/sumstat/PCC_AC-DLPFC.EZ.V_simple.rds'}



V = readRDS(resid_cor)
prior = readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Meta_analysis/../Fine_Mapping/Mixture_Prior/PCC_AC-DLPFC.ed_bovy.V_simple.rds')
print(paste("Number of components in the mixture prior:", length(prior$U)))
prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U),
                                    include_indices = get_prior_indices(dat$Z, prior$U[[1]]),
                                    max_mixture_len=-1)
if("original" == 'original'){
    R = readRDS(dat$ld.file$ld.path)
}else if("original" == 'remove_cov'){
    R = dat$LD
}

# Remove the NA SNPs in R
K = na.omit(R)
K = K[,which(colnames(K)%in%rownames(K))]
R = K

## Remove duplicated in Z
dat$Z = dat$Z[!duplicated(dat$snps),]
dat$snps = dat$snps[!duplicated(dat$snps)]

# Retaining only the overlapping snps

R = R[which(rownames(R)%in%dat$snps),which(colnames(R)%in%dat$snps)]
dat$Z = dat$Z[which(dat$snps%in%rownames(R)),]
dat$snps = dat$snp[which(dat$snps%in%rownames(R))]


## Initiate the computation
st = proc.time()
mv_res = mvsusieR::mvsusie_rss(dat$Z, R, L=10,
                               prior_variance=prior, residual_variance=V,
                               precompute_covariances=T, compute_objective=T,
                               estimate_prior_variance=T, estimate_prior_method='EM',
                               max_iter = 10, n_thread=1)
mv_res$time = proc.time() - st
#if(mv_res$convergence$converged == FALSE){
#    stop('Fail to converge.')
#}
mv_res$cs_corr = susieR:::get_cs_correlation(mv_res, Xcorr=R)

# Get list of cs snps
mv_output = mv_res

saveRDS(mv_res, '/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Fine_mapping/ENSG00000010818.LDoriginal.V_simple.mvsusierss.model.rds')

[1] "Number of components in the mixture prior: 13"


Initializing prior object ...

Number of components in the mixture prior: 9

Warning message in mvsusie_core(data, s_init, L, prior_variance, prior_weights, :
“precompute_covariances option is disabled when prior variances are to be updated.”
Running IBSS algorithm ...

Iteration 1 delta = Inf

Iteration 2 delta = 93.4710739815798

Iteration 3 delta = 111.498303332209

Iteration 4 delta = 104.816309294962

Iteration 5 delta = 87.8519439695206

Iteration 6 delta = 73.7493626465172

Iteration 7 delta = 60.8469097495259

Iteration 8 delta = 95.5075010571134

Iteration 9 delta = 81.7734867497367

Warning message in SuSiE_model$fit(data, prior_weights, estimate_prior_method, check_null_threshold, :
“IBSS failed to converge after 10 iterations. Perhaps you should increase max_iter and try again.”
Iteration 10 delta = 76.8347776862083



In [ ]:
mv_output$

In [6]:
dat$ld.file$ld.path

[1] "/mnt/mfs/statgen/neuro-twas/mv_wg/cache_arch/cache/geneTpmResidualsAgeGenderAdj_rename.ENSG00000010818.merged.ld.rds"

In [25]:
get_prior_indices <- function(Z, U) {
  # make sure the prior col/rows match the colnames of the Y matrix
  z_names = colnames(Z)
  u_names = colnames(U)
  if (is.null(z_names) || is.null(u_names)) {
      return(NULL)
  } else if (identical(z_names, u_names)) {
      return(NULL)
  } else {
      return(match(z_names, u_names))
  }
}

library(mvsusieR)
library("dplyr")
library("tibble")
library("purrr")
library("readr")
library("tidyr")
dat = readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Fine_mapping/preprocessed/ENSG00000007944.rds')
gene_id = "ENSG00000007944"




if (FALSE == TRUE){
  resid_cor = paste(c("/mnt/mfs/statgen/neuro-twas/TWAS_sumstat/residcor_2/",gene_id),collapse = "")
  }else{resid_cor = '/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Meta_analysis/sumstat/PCC_AC-DLPFC.EZ.V_simple.rds'}



V = readRDS(resid_cor)
prior = readRDS('/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Meta_analysis/../Fine_Mapping/Mixture_Prior/PCC_AC-DLPFC.ed_bovy.V_simple.rds')
print(paste("Number of components in the mixture prior:", length(prior$U)))
prior = mvsusieR::create_mash_prior(mixture_prior=list(weights=prior$w, matrices=prior$U),
                                    include_indices = get_prior_indices(dat$Z, prior$U[[1]]),
                                    max_mixture_len=-1)
if("original" == 'original'){
    R = readRDS(dat$ld.file$ld.path)
}else if("original" == 'remove_cov'){
    R = dat$LD
}

## Remove the NA SNPs in R, assuming all NA = 0
R[is.na(R)] = 0
#K = K[,which(colnames(K)%in%rownames(K))]
#R = K
##
### Remove duplicated in Z
dat$Z = dat$Z[!duplicated(dat$snps),]
dat$snps = dat$snps[!duplicated(dat$snps)]

# Retaining only the overlapping snps

R = R[which(rownames(R)%in%dat$snps),which(colnames(R)%in%dat$snps)]
dat$Z = dat$Z[which(dat$snps%in%rownames(R)),]
dat$snps = dat$snp[which(dat$snps%in%rownames(R))]


## Initiate the computation
st = proc.time()
mv_res = mvsusieR::mvsusie_rss(dat$Z, R, L=10,
                               prior_variance=prior, residual_variance=V,
                               precompute_covariances=T, compute_objective=T,
                               estimate_prior_variance=T, estimate_prior_method='EM',
                               max_iter = 10, n_thread=1)
mv_res$time = proc.time() - st
#if(mv_res$convergence$converged == FALSE){
#    stop('Fail to converge.')
#}
mv_res$cs_corr = susieR:::get_cs_correlation(mv_res, Xcorr=R)

# Get list of cs snps
mv_output = mv_res

saveRDS(mv_res, '/mnt/mfs/statgen/xqtl_workflow_testing/testing_10/Fine_mapping/ENSG00000007944.LDoriginal.V_simple.mvsusierss.model.rds')

[1] "Number of components in the mixture prior: 13"


Initializing prior object ...

Number of components in the mixture prior: 9

Warning message in mvsusie_core(data, s_init, L, prior_variance, prior_weights, :
“precompute_covariances option is disabled when prior variances are to be updated.”
Running IBSS algorithm ...

Iteration 1 delta = Inf

Iteration 2 delta = 36.8710467768473

Iteration 3 delta = 62.2394599501906

Iteration 4 delta = 76.5985317322887

Iteration 5 delta = 76.0297706618139

Iteration 6 delta = 77.0063559574114

Iteration 7 delta = 84.5405722383352

Iteration 8 delta = 83.6072487486859

Iteration 9 delta = 80.0269601131345

Warning message in SuSiE_model$fit(data, prior_weights, estimate_prior_method, check_null_threshold, :
“IBSS failed to converge after 10 iterations. Perhaps you should increase max_iter and try again.”
Iteration 10 delta = 76.5780019188679



In [20]:
R[is.na(R)] = 0

In [22]:
na.omit(R)

,6:15629223_C_T,6:15629293_T_TA,6:15629784_A_ATTTAAG,6:15629838_C_T,6:15629844_C_T,6:15630917_A_G,6:15631104_A_G,6:15631687_G_GAC,6:15632741_C_T,6:15632813_A_G,...,6:16645228_A_G,6:16645278_C_T,6:16645599_A_G,6:16645643_C_T,6:16646478_A_C,6:16647158_A_G,6:16647646_C_T,6:16647991_A_G,6:16648055_C_T,6:16648072_C_T
6:15629223_C_T,1.0000000,-0.30055000,-0.30055000,0.9988850,-0.30055000,-0.21590800,-0.4990780,-0.2257790,0.1176010,-0.30220700,...,-0.01377900,0.01758350,0.01758350,-0.01073220,-0.011321900,0.01075200,0.000200045,-0.017308300,-0.000733816,-0.021255100
6:15629293_T_TA,-0.3005500,1.00000000,1.00000000,-0.3015380,1.00000000,-0.00608397,-0.1722400,-0.0702045,0.5708340,0.99299800,...,-0.02535920,0.01120590,0.01120590,-0.03566680,0.005641360,-0.01179430,0.030179500,0.000793227,0.005063510,0.011737600
6:15629784_A_ATTTAAG,-0.3005500,1.00000000,1.00000000,-0.3015380,1.00000000,-0.00608397,-0.1722400,-0.0702045,0.5708340,0.99299800,...,-0.02535920,0.01120590,0.01120590,-0.03566680,0.005641360,-0.01179430,0.030179500,0.000793227,0.005063510,0.011737600
6:15629838_C_T,0.9988850,-0.30153800,-0.30153800,1.0000000,-0.30153800,-0.21657800,-0.4981440,-0.2265590,0.1165670,-0.30319100,...,-0.01402030,0.01736370,0.01736370,-0.00900598,-0.010942900,0.01077820,-0.001336770,-0.017191900,-0.000553454,-0.021086900
6:15629844_C_T,-0.3005500,1.00000000,1.00000000,-0.3015380,1.00000000,-0.00608397,-0.1722400,-0.0702045,0.5708340,0.99299800,...,-0.02535920,0.01120590,0.01120590,-0.03566680,0.005641360,-0.01179430,0.030179500,0.000793227,0.005063510,0.011737600
6:15630917_A_G,-0.2159080,-0.00608397,-0.00608397,-0.2165780,-0.00608397,1.00000000,-0.1214970,-0.0463222,-0.0684518,-0.00453195,...,0.09322680,0.01363420,0.01363420,0.00538200,0.022218900,0.00855187,-0.029145700,0.035773500,-0.021045600,0.034008700
6:15631104_A_G,-0.4990780,-0.17224000,-0.17224000,-0.4981440,-0.17224000,-0.12149700,1.0000000,-0.1648520,-0.2759990,-0.16865600,...,-0.02666350,-0.00226357,-0.00226357,-0.02794200,-0.010585700,0.00202796,0.019246900,-0.005988570,-0.002626830,-0.005770390
6:15631687_G_GAC,-0.2257790,-0.07020450,-0.07020450,-0.2265590,-0.07020450,-0.04632220,-0.1648520,1.0000000,-0.1218600,-0.06859870,...,-0.00549391,-0.00576802,-0.00576802,0.03656580,-0.027253400,-0.04515320,0.022869700,-0.009319990,0.034670900,-0.021664400
6:15632741_C_T,0.1176010,0.57083400,0.57083400,0.1165670,0.57083400,-0.06845180,-0.2759990,-0.1218600,1.0000000,0.56358500,...,-0.00955204,0.00254918,0.00254918,-0.03874440,-0.010174400,-0.02992200,0.038561000,-0.029596500,0.031007300,-0.009449680
6:15632813_A_G,-0.3022070,0.99299800,0.99299800,-0.3031910,0.99299800,-0.00453195,-0.1686560,-0.0685987,0.5635850,1.00000000,...,-0.02467120,0.01210460,0.01210460,-0.03194430,0.008302090,-0.00792315,0.023332800,0.004294530,0.000429141,0.015114400
